# BikewaySim

Use this jupyter notebook to run BikewaySim. BikewaySim finds the shortest path between any origin and destination using Dijkstra's algorithm. 

BikewaySim requires the following before running:

- A nodes shapefile/geojson with node ids.
- A links shapefile/geojson with refrences ids (i.e. column that specifies the starting and ending node id)
- A csv with origin and destination pairs

Notes:
- Run the convert_to_bikewaysim_network script before running bikewaysim to make sure your network is compatible
- The desired link cost should be specifed in the "distance" column of the links. If you want to specify your own link costs using link attributes, then modify the "distance" column before using the network in BikewaySim

Other Notes:

Use Python 3.7+


The following third-party packages are used in the BikewaySim module:
```python
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

import networkx as nx

import matplotlib.pyplot as plt
```

# Preparation

Import modules/packages required for this notebook.

In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpdui

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/%autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
pd.options.display.max_columns = None  # display all columns

In [2]:
# import TransportSim module
from core_script.getPaths_transportSim import *

### Setting the directory

In [9]:
# need to set this environmental path for network data and query data at separate locations
user_directory = os.fspath(Path.home()) #get home directory and convert to path string

# set path variable for BikewaySim
os.environ['PROJ_LIB'] = user_directory + "/Documents/GitHub/BikewaySimDev"

# network shapefile data path directory
os.environ['bws_NETWORK'] = r"C:\Users\tpassmore6\Documents\BikewaySimData\processed_shapefiles\prepared_network"

# OD Pairs directory
os.environ['bws_TRIPS'] = os.environ['PROJ_LIB'] + "/trips_bws"

# create BikewaySim object for running 'bikewaysim' option
bws = BikewaySim(os.environ['PROJ_LIB'], os.environ['bws_NETWORK'], os.environ['bws_TRIPS'])

Initialize BikewaySim object to process sidewalk graph and pre-process trip queries!
Set/Update BikewaySim dict_settings!


### Prepare networks for BikewaySim

In [10]:
# prepare networks
df_links, dict_bike = bws.prepare_network(grid_size=10000)
# print default settings

** Initialize BikewaySim network **
It takes 15.778 seconds to run the function 'prepare_network()'!


In [11]:
# strategy: 1. given origin time find earliest arrival
#           2. given expected arrival time find latest departure time
dict_settings = {'walk_speed': 2.0,  # people's walking speed 2 mph
                 'grid_size': 10000.0,  # for searching nearby links by grouping links to grids with width 10000 ft. for efficiency in searching
                 'ntp_dist_thresh': 5280.0,  # node to point (maximum distance access to network from origin/destination); (walking) distance threshold
                 'network': {'bike': dict_bike},  # dump in networks and modes
                 # strategy determines network link's direction.
                 # Strategy 1: Find earliest arrival given query time as departure time
                 # Strategy 2: Find latest departure time given query time as arrival time
             
                 'strategy': {'bike': 1},  # 1. find earliest arrival 2. find latest departure
                 'query_time': [8],  # departure time or arrival time of a trip, depends on the strategy
                 
                 'walk_thresh': {'bike': 0.1},  # walking threshold is 0.1 miles
                 'num_options': {'bike': 1},  # if set to 2, return 2-shortest paths
                 'plot_all': False,  # if True, plot results and save plots for all routes found #this is broken now
                 'one_by_one': False  # set time and strategy one by one
                 }
# load dict_settings to the sws object
bws.dict_settings = dict_settings

Set/Update BikewaySim dict_settings!


In [12]:
df_points = bws.prepare_trips(option='bike')
display(df_points.head())

** load trip data & prepare sample **
It takes 0.310 seconds to run the function 'prepare_trips()'!


,trip_id,ori_lat,ori_lon,dest_lat,dest_lon,geometry,ox,oy,ox_sq,oy_sq,o_node,o_t,o_d,dx,dy,dx_sq,dy_sq,d_node,d_t,d_d
0,565_496,33.780671,-84.370917,33.783959,-84.39495,POINT (2227222.729 1376527.138),2.234522e+06,1.375315e+06,223.0,138.0,10299693,0.038204,0.076407,2.227223e+06,1.376527e+06,223.0,138.0,201181586757,0.011415,0.022829


In [15]:
# run all queries
bws.run_all()

**for each trip, find k-shortest paths**
It takes 0.007 seconds to run the function 'run_all()'!


# Results

The trip inputs & results are stored in ' ***/trips_bws/results*** '

### Columns defintion of the output results
Every row correpsonds to a link<br>
**A**: origin node id<br>
**B**: destination node id<br>
**dist**: distances<br>
**mode**: mode of transportation (e.g., walk, bike, etc.)<br>
**strategy**: chosen strategy<br>
**route_num**: the k number of k-shortest routes return for each link<br>
**sequence**: the trip segment number in a sequence of trips<br>
**time**: time it takes to travel through the link<br>
**timeStamp**: accumulative time of the current trip (in hours)<br>
**trip_id**: trip id (from the trip query)<br>
**route**: if there is a crosswalk, which crosswalk it uses<br>

### For graph outputs

***BikeSim/build_graph/results_routes***

naming rule:

{trip_id}_{(k-1)th route}.PNG

e.g.,

{4001880_1_1}_{0}.PNG